<a href="https://colab.research.google.com/github/AndreaBur/AI_Implementation/blob/main/CLASIFICACIONJERARQUICA_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**MODELO DE CLASIFICACION JERARQUICA**

La clasificación jerárquica es más compleja que los tipos tradicionales de clasificación, ya que las clases están organizadas en niveles o árboles.

Aunque scikit-learn no incluye directamente soporte nativo para clasificación jerárquica, sí podemos simularla con una estrategia de clasificadores encadenados por niveles (por ejemplo: Reino → Filo → Clase).

In [ ]:
# Paso 1: Importar librerías
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd

# Paso 2: Crear datos sintéticos para la jerarquía
# Nivel 1: Clase general (0 = Mamífero, 1 = Ave)
X, y_general = make_classification(n_samples=1000, n_features=5, n_classes=2, random_state=42)

# Crear etiquetas del segundo nivel en función del primer nivel
# Mamíferos: clase 0 → [0 = Perro, 1 = Gato]
# Aves: clase 1 → [2 = Loro, 3 = Águila]
y_specific = []

np.random.seed(42)
for label in y_general:
    if label == 0:
        y_specific.append(np.random.choice([0, 1]))  # Perro o Gato
    else:
        y_specific.append(np.random.choice([2, 3]))  # Loro o Águila

y_specific = np.array(y_specific)

# Paso 3: Dividir los datos
X_train, X_test, y_general_train, y_general_test, y_specific_train, y_specific_test = train_test_split(
    X, y_general, y_specific, test_size=0.3, random_state=42
)

# Paso 4: Entrenar clasificador para nivel 1 (Clase general)
clf_general = DecisionTreeClassifier()
clf_general.fit(X_train, y_general_train)

# Paso 5: Entrenar clasificadores separados para cada clase del primer nivel
# Mamíferos
mask_mamifero = y_general_train == 0
clf_mamifero = DecisionTreeClassifier()
clf_mamifero.fit(X_train[mask_mamifero], y_specific_train[mask_mamifero])

# Aves
mask_ave = y_general_train == 1
clf_ave = DecisionTreeClassifier()
clf_ave.fit(X_train[mask_ave], y_specific_train[mask_ave])

# Paso 6: Predicción jerárquica
y_pred_general = clf_general.predict(X_test)
y_pred_specific = []

for i, pred_gen in enumerate(y_pred_general):
    if pred_gen == 0:
        pred = clf_mamifero.predict([X_test[i]])
    else:
        pred = clf_ave.predict([X_test[i]])
    y_pred_specific.append(pred[0])

# Paso 7: Evaluación
print("Reporte Nivel 1 (Clase General):")
print(classification_report(y_general_test, y_pred_general, target_names=["Mamífero", "Ave"]))

print("\nReporte Nivel 2 (Subtipos):")
etiquetas = ["Perro", "Gato", "Loro", "Águila"]
print(classification_report(y_specific_test, y_pred_specific, target_names=etiquetas))


Reporte Nivel 1 (Clase General):
              precision    recall  f1-score   support

    Mamífero       0.88      0.82      0.84       146
         Ave       0.84      0.89      0.86       154

    accuracy                           0.85       300
   macro avg       0.86      0.85      0.85       300
weighted avg       0.85      0.85      0.85       300


Reporte Nivel 2 (Subtipos):
              precision    recall  f1-score   support

       Perro       0.49      0.39      0.44        74
        Gato       0.49      0.53      0.51        72
        Loro       0.32      0.39      0.35        70
      Águila       0.41      0.39      0.40        84

    accuracy                           0.42       300
   macro avg       0.43      0.42      0.42       300
weighted avg       0.43      0.42      0.42       300

